In [1]:
from flask import Flask, render_template, request, jsonify
from pymongo import MongoClient
import hashlib
import torch
import globalvar
import time
import requests
import sys
sys.path.append("./predict/")
import predict_after
import os
import datetime
import shutil

In [2]:
traitsList = ['ALL',
 'MG',
 'ST',
 'FC',
 'P_CLR',
 'P_FRM',
 'P_DENS',
 'POD',
 'SC_L',
 'SC_CLR',
 'H_CLR',
 'protein',
 'oil',
 'Linoleic',
 'Linolenic',
 'eno',
 'R1',
 'R8',
 'Hgt',
 'Mat',
 'Ldg',
 'SQ',
 'SdWgt',
 'Yield']
filePath = ""
fileName = ""

In [3]:
app = Flask(__name__)
@app.route('/')
def index():
    return render_template('test.html')

@app.route('/Document')
def document():
    return render_template('Document.html')

@app.route('/AboutUs')
def AboutUs():
    return render_template('AboutUs.html')

@app.route('/Search')
def Search():
    return render_template('Search.html')

@app.route('/UploadData')
def Predict():
    return render_template('UploadData.html')

@app.route('/submit', methods=['GET', 'POST'])
def submit():
    if request.method == 'POST':
        ID = request.form['inputText']
        traits = request.form.getlist('options')
        if len(ID) and len(traits) != 0:
            if traits[0] == 'all':
                traits = traits[1:]
            traitsNames = [traitsList[int(i)] for i in traits]
            client = MongoClient()
            db = client.test
            collection = db.test
            rets = collection.find({'acid': ID})
            results = []
            for i in rets:
                for j in traitsNames:
                    value = i.get(j, 'No result')
                    result = {"trait": j, "value":value}
                    results.append(result)
        else:
            results = False
    return render_template('/Search.html', results=results, ID=ID)

@app.route('/upload', methods=['POST'])
def upload():
    file = request.files.get('file')
    content = file.read()  # 从缓存中被拿出，用byte保存
    if content:
        file_md5 = hashlib.md5(content).hexdigest() # 计算 MD5
        date = str(datetime.datetime.now()).split(' ')
        newfilename = date[0] + '-' + date[1].replace(':', '.') + '-' + file_md5
        if not os.path.exists(f"./{newfilename}"):
            os.mkdir(f"./{newfilename}")  # 创建文件夹
        savepath = f"./{newfilename}/" + file.filename  # 组合出路径
        global fileName
        fileName = file.filename
        global filePath 
        filePath = f"./{newfilename}/"
        client = MongoClient()
        db = client.files
        result = db.files.find_one({'md5': file_md5}) # 查询 MongoDB
        if not result:
            with open(savepath, 'wb') as f:
                f.write(content)
            db.files.insert_one({'path':savepath, 'md5': file_md5})
        else:
            print(result['path'])
            shutil.copyfile(result['path'], savepath)
        return jsonify({'errno':0, 'errmsg':'success'})
    else:
        return jsonify({'errno':1, 'errmsg':'file is empty'})

@app.route('/progress')
def update_progress():
    progress = globalvar.getProgressBar()  # 获取当前进度
    return jsonify(progress=progress)  # 返回进度信息
    
@app.route('/Predict', methods=['POST'])
def predict_():
    globalvar.initProgressBar()
    traits = request.form.getlist('options')
    if len(traits) != 0:
        if traits[0] != 'all':
            traitsNames = [traitsList[int(i)] for i in traits]
            print(traitsNames)
            predict_after.predict(filePath + fileName, traitsNames, filePath, if_all=False)
            
        else:
            traits = traits[1:]
            traitsNames = [traitsList[int(i)] for i in traits]
            predict_after.predict(filePath + fileName, [], filePath, if_all=True)
    return list(y_hat)



if __name__ == '__main__':
    app.run()  # 实时更改

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [19/May/2023 11:11:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:11:59] "GET /static/bootstrap.min.css HTTP/1.1" 304 -
127.0.0.1 - - [19/May/2023 11:11:59] "GET /static/bootstrap.bundle.min.js HTTP/1.1" 304 -
127.0.0.1 - - [19/May/2023 11:12:00] "GET /UploadData HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:00] "GET /static/bootstrap.min.css HTTP/1.1" 304 -
127.0.0.1 - - [19/May/2023 11:12:00] "GET /static/fileinput.min.css HTTP/1.1" 304 -
127.0.0.1 - - [19/May/2023 11:12:00] "GET /static/fileinput.min.js HTTP/1.1" 304 -
127.0.0.1 - - [19/May/2023 11:12:00] "GET /static/bootstrap.bundle.min.js HTTP/1.1" 304 -
127.0.0.1 - - [19/May/2023 11:12:00] "GET /img/loading.gif HTTP/1.1" 404 -
127.0.0.1 - - [19/May/2023 11:12:01] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:02] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:03] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/M

./2023-05-19-10.34.58.836402-21656b0cb3b93d93d66473c3bca94cb1test_sample.vcf


127.0.0.1 - - [19/May/2023 11:12:06] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:07] "GET /progress HTTP/1.1" 200 -


['protein']


127.0.0.1 - - [19/May/2023 11:12:08] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:09] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:10] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:11] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:12] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:13] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:14] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:15] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:16] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:17] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:18] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:19] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:20] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:21] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:22] "GET /progr

df shape (2169, 42195)


127.0.0.1 - - [19/May/2023 11:12:49] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:50] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:50] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:51] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:52] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:53] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:55] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:55] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:56] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:57] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:58] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:12:59] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:13:00] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:13:02] "GET /progress HTTP/1.1" 200 -
127.0.0.1 - - [19/May/2023 11:13:02] "GET /progr

dataset already completed!
2169
Data process has done! Use time:84.8796558380127 
 Start data predict


[2023-05-19 11:13:34,067] ERROR in app: Exception on /Predict [POST]
Traceback (most recent call last):
  File "D:\anaconda3\envs\opencv_cuda\lib\site-packages\flask\app.py", line 2190, in wsgi_app
    response = self.full_dispatch_request()
  File "D:\anaconda3\envs\opencv_cuda\lib\site-packages\flask\app.py", line 1486, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "D:\anaconda3\envs\opencv_cuda\lib\site-packages\flask\app.py", line 1484, in full_dispatch_request
    rv = self.dispatch_request()
  File "D:\anaconda3\envs\opencv_cuda\lib\site-packages\flask\app.py", line 1469, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\Users\PinkFloyd\AppData\Local\Temp\ipykernel_9780\323447806.py", line 87, in predict_
    predict_after.predict(filePath + fileName, traitsNames, filePath, if_all=False)
  File "D:\Projects\website\soybean\./predict\predict_after.py", line 46, in __init__
    self.forward()
  File 